In [140]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [141]:
# Import necessary libraries
import pandas as pd
import numpy as np
from IPython.display import display
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WIA1007_Group_Assignment/laptopData_table_cleaned.csv')
df.head()

,Company,TypeName,Inches,Ram,OpSys,Weight,Price,Touchscreen,Ips,Screen_Width,Screen_Height,ppi,Processor,CPU_Speed,SSD,HDD,Flash,Hybrid,Gpu_Brand,OS
0,Apple,Ultrabook,13.3,8,macOS,1.37,71378.6832,0,1,2560,1600,226.983005,Intel Core i5,2.3,128,0,0,0,Intel,Mac
1,Apple,Ultrabook,13.3,8,macOS,1.34,47895.5232,0,0,1440,900,127.677940,Intel Core i5,1.8,0,0,128,0,Intel,Mac
2,HP,Notebook,15.6,8,No OS,1.86,30636.0000,0,0,1920,1080,141.211998,Intel Core i5,2.5,256,0,0,0,Intel,Other OS/No OS/Linux
3,Apple,Ultrabook,15.4,16,macOS,1.83,135195.3360,0,1,2880,1800,220.534624,Intel Core i7,2.7,512,0,0,0,AMD,Mac
4,Apple,Ultrabook,13.3,8,macOS,1.37,96095.8080,0,1,2560,1600,226.983005,Intel Core i5,3.1,256,0,0,0,Intel,Mac


In [142]:
# Format ppi to two decimal places
df['ppi'] = df['ppi'].round(2)

# Change price from rupees to USD and format the price column to two decimal places
df['Price'] = (df['Price'] * 0.01176).round(2)

# Check if there is any row with Weight equal to 0.0002
if (df['Weight'] == 0.0002).any():
    # Get the index of the row
    weight_row_index = df[df['Weight'] == 0.0002].index[0]

    # Drop the row at the found index
    df = df.drop(index=weight_row_index)

In [145]:
# Table 1: Data Properties
table_1 = []

for column in df.columns:
    col_data = df[column]
    col_type = col_data.dtypes
    unique_values = col_data.nunique()
    null_values = col_data.isnull().sum()
    min_val = round(col_data.min(), 2) if col_type != 'object' else '-'
    max_val = round(col_data.max(), 2) if col_type != 'object' else '-'

    # Determine types and levels
    if col_type == 'object':
        data_type = "String"
        measurement_level = "Nominal"
    elif np.issubdtype(col_type, np.integer):
        data_type = "int"
        measurement_level = "Ratio"
        IQR = col_data.quantile(0.75) - col_data.quantile(0.25)
        lower_bound = col_data.quantile(0.25) - 1.5 * IQR
        upper_bound = col_data.quantile(0.75) + 1.5 * IQR
    elif np.issubdtype(col_type, np.floating):
        data_type = "float"
        measurement_level = "Ratio"
        IQR = col_data.quantile(0.75) - col_data.quantile(0.25)
        lower_bound = col_data.quantile(0.25) - 1.5 * IQR
        upper_bound = col_data.quantile(0.75) + 1.5 * IQR
    else:
        data_type = "Unknown"
        measurement_level = "Unknown"

    units = {
        "Inches": "inch",
        "Ram": "Gb",
        "Weight": "kg",
        "Price": "USD",
        "Screen_Width": "pixel",
        "Screen_Height": "pixel",
        "ppi": "ppi",
        "CPU_Speed": "GHz",
        "SSD": "Gb",
        "HDD": "Gb",
        "Flash": "Gb",
        "Hybrid": "Gb",
        "GPU_Speed": "GHz"
    }

    # Append data properties to table_1
    table_1.append({
        "Variable": column,
        "Types of Data": "Numeric" if data_type in ["int", "float"] else "Categorical",
        "Data Types": data_type,
        "Measurement Level": measurement_level,
        "Units": units.get(column, "-"),
        "Range": f"{min_val} - {max_val}" if min_val != '-' and max_val != '-' else "-",
        "Min Value": min_val,
        "Top Value": max_val,
        "Unique Values": unique_values,
        "Null Values": null_values,
        "Outliers": f"Yes, < {lower_bound:.2f} and > {upper_bound:.2f}" if col_type != "object" and (lower_bound != 0 or upper_bound != 0) and (min_val < lower_bound or max_val > upper_bound) else "No"
    })

table_1_df = pd.DataFrame(table_1)
print("\nTable 1: Data Properties")
display(table_1_df)

# Save the tables to CSV for submission
table_1_df.to_csv("/content/drive/MyDrive/Colab Notebooks/WIA1007_Group_Assignment/Table_1_Data_Properties.csv", index=False)


Table 1: Data Properties


,Variable,Types of Data,Data Types,Measurement Level,Units,Range,Min Value,Top Value,Unique Values,Null Values,Outliers
0,Company,Categorical,String,Nominal,-,-,-,-,19,0,No
1,TypeName,Categorical,String,Nominal,-,-,-,-,6,0,No
2,Inches,Numeric,float,Ratio,inch,10.1 - 35.6,10.1,35.6,24,0,"Yes, < 11.60 and > 18.00"
3,Ram,Numeric,int,Ratio,Gb,1 - 64,1,64,10,0,"Yes, < -2.00 and > 14.00"
4,OpSys,Categorical,String,Nominal,-,-,-,-,9,0,No
5,Weight,Numeric,float,Ratio,kg,0.69 - 11.1,0.69,11.1,180,0,"Yes, < 0.27 and > 3.55"
6,Price,Numeric,float,Ratio,USD,109.02 - 3821.47,109.02,3821.47,766,0,"Yes, < -460.62 and > 1768.55"
7,Touchscreen,Numeric,int,Ratio,-,0 - 1,0,1,2,0,No
8,Ips,Numeric,int,Ratio,-,0 - 1,0,1,2,0,No
9,Screen_Width,Numeric,int,Ratio,pixel,1366 - 3840,1366,3840,13,0,"Yes, < 1120.00 and > 2400.00"


In [144]:
from IPython.core.display import display, HTML

# Table 2: Statistics
table_2 = []

for column in df.columns:
    col_data = df[column].dropna()  # Drop null values

    # Check for numeric columns
    if df[column].dtype in ['int64', 'float64']:
        mean = round(col_data.mean(), 1)
        median = round(col_data.median(), 1)
        mode = col_data.mode().iloc[0] if not col_data.mode().empty else None
        std_dev = round(col_data.std(), 1)
        variance = round(col_data.var(), 1)
        skewness = round(col_data.skew(), 1)
        kurtosis = round(col_data.kurt(), 1)
        completeness = f"{(1 - col_data.isnull().mean()) * 100:.2f}%"
        percentiles = col_data.quantile([0.25, 0.5, 0.75]).values  # Calculate percentiles
        percentiles_str = f"{round(percentiles[0], 1):.1f}, {round(percentiles[1], 1):.1f}, {round(percentiles[2], 1):.1f}"
        frequency = "-"  # Frequency is not applicable for numeric columns

        if column in ["Touchscreen", "Ips"]:
            percentiles_str = '-'
            median = '-'
            std_dev = '-'
            variance = '-'
            skewness = '-'
            kurtosis = '-'
            value_counts = col_data.value_counts(normalize=True) * 100  # Normalize to get percentages
            frequency = "<br>".join([f"{key}: {round(value, 1):.1f}%" for key, value in value_counts.items()])

    else:  # Handle non-numeric columns
        mean = "-"
        median = "-"
        mode = col_data.mode().iloc[0] if not col_data.mode().empty else None
        std_dev = "-"
        variance = "-"
        skewness = "-"
        kurtosis = "-"
        completeness = f"{(1 - col_data.isnull().mean()) * 100:.2f}%"
        percentiles_str = "-"  # Percentiles do not apply to non-numeric columns

        # Calculate frequency as unique values and their percentages
        value_counts = col_data.value_counts(normalize=True) * 100  # Normalize to get percentages
        frequency = "\n".join([f"{key}: {round(value, 1):.1f}%" for key, value in value_counts.items()])

    # Append statistical properties to table_2
    table_2.append({
        "Variable": column,
        "Frequency": frequency,
        "Percentile (25th, 50th, 75th)": percentiles_str,
        "Data Completeness": completeness,
        "Mean": mean,
        "Median": median,
        "Mode": mode,
        "Standard Deviation": std_dev,
        "Variance": variance,
        "Skewness": skewness,
        "Kurtosis": kurtosis
    })

# Convert table_2 into a DataFrame
table_2_df = pd.DataFrame(table_2)

# Save the table to a CSV file
table_2_df.to_csv("/content/drive/MyDrive/Colab Notebooks/WIA1007_Group_Assignment/Table_2_Statistics.csv", index=False)

# Use HTML to display the DataFrame with line breaks in Google Colab
table_2_html = table_2_df.to_html(escape=False).replace("\\n", "<br>")
display(HTML(table_2_html))


,Variable,Frequency,"Percentile (25th, 50th, 75th)",Data Completeness,Mean,Median,Mode,Standard Deviation,Variance,Skewness,Kurtosis
0,Company,Lenovo: 22.8%Dell: 22.4%HP: 20.9%Asus: 12.3%Acer: 8.1%MSI: 4.2%Toshiba: 3.7%Apple: 1.7%Samsung: 0.7%Razer: 0.6%Mediacom: 0.6%Microsoft: 0.5%Xiaomi: 0.3%Vero: 0.3%Chuwi: 0.2%Google: 0.2%LG: 0.2%Huawei: 0.2%Fujitsu: 0.2%,-,100.00%,-,-,Lenovo,-,-,-,-
1,TypeName,Notebook: 55.8%Gaming: 16.0%Ultrabook: 15.0%2 in 1 Convertible: 9.1%Workstation: 2.2%Netbook: 1.9%,-,100.00%,-,-,Notebook,-,-,-,-
2,Inches,-,"14.0, 15.6, 15.6",100.00%,15.1,15.6,15.6,2.0,3.8,4.1,39.0
3,Ram,-,"4.0, 8.0, 8.0",100.00%,8.5,8.0,8,5.6,31.0,3.6,25.7
4,OpSys,Windows 10: 82.2%No OS: 5.0%Linux: 4.8%Windows 7: 3.5%Chrome OS: 2.1%macOS: 1.0%Mac OS X: 0.6%Windows 10 S: 0.6%Android: 0.1%,-,100.00%,-,-,Windows 10,-,-,-,-
5,Weight,-,"1.5, 2.0, 2.3",100.00%,2.1,2.0,2.2,0.8,0.7,3.1,21.3
6,Price,-,"375.3, 612.8, 932.6",100.00%,704.6,612.8,688.6,438.9,192624.4,1.5,4.4
7,Touchscreen,0: 85.4%1: 14.6%,-,100.00%,0.1,-,0,-,-,-,-
8,Ips,0: 71.7%1: 28.3%,-,100.00%,0.3,-,0,-,-,-,-
9,Screen_Width,-,"1600.0, 1920.0, 1920.0",100.00%,1894.2,1920.0,1920,494.8,244805.1,2.2,6.4
